In [35]:
# Auto-reload setup
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import sys
import os
sys.path.append(os.path.abspath('../src'))
from data_processing import *
print('Successful import')

Successful import


In [37]:
folder = '../data/raw'
types = ['prices_raw', 'total_load_day_ahead_raw', 'generation_production_raw']

grouped = group_files_by_type(folder, types, extension='.csv')
dfs = load_dataframes(grouped)

# Access each DataFrame
df_prices = dfs['prices_raw']
df_load = dfs['total_load_day_ahead_raw']
df_gen = dfs['generation_production_raw']
print(len(df_prices))

4


In [38]:
df_load = [drop_unecessary_columns(df, 'Day-ahead Total Load Forecast [MW] - BZN|FR') for df in df_load] 
df_load = [rename_and_reoder_columns(df, None, ['Date', 'Total Load']) for df in df_load]
df_load = [setup_time(df, 'Date', '%d.%m.%Y %H:%M') for df in df_load]
df_load = [fill_hourly_nans_by_rolling_mean(df, 'Date', 'Total Load') for df in df_load]

df = concat_data(df_load)
df_summary(df)
df.to_csv('../data/clean/exploitable/Concat_load.csv', index=False)

OUI
OUI
OUI
OUI
Shape: (35064, 2)

Column Types:
Date          datetime64[ns]
Total Load           float64
dtype: object

Missing Values:
Date          0
Total Load    0
dtype: int64

First Rows:
                 Date  Total Load
0 2021-01-01 00:00:00     65761.0
1 2021-01-01 01:00:00     64139.0
2 2021-01-01 02:00:00     62657.0
3 2021-01-01 03:00:00     59481.0
4 2021-01-01 04:00:00     57656.0

Last Rows:
                     Date  Total Load
35059 2024-12-31 19:00:00     68220.0
35060 2024-12-31 20:00:00     65266.0
35061 2024-12-31 21:00:00     62886.0
35062 2024-12-31 22:00:00     63983.0
35063 2024-12-31 23:00:00     64603.0
----------------------------------------


In [39]:
df_gen = [drop_unecessary_columns(df, ["Area","Biomass - Actual Aggregated [MW]","Energy storage - Actual Aggregated [MW]","Fossil Brown coal/Lignite - Actual Aggregated [MW]","Fossil Coal-derived gas - Actual Aggregated [MW]","Fossil Gas - Actual Aggregated [MW]","Fossil Hard coal - Actual Aggregated [MW]","Fossil Oil - Actual Aggregated [MW]","Fossil Oil shale - Actual Aggregated [MW]","Fossil Peat - Actual Aggregated [MW]","Geothermal - Actual Aggregated [MW]","Hydro Pumped Storage - Actual Aggregated [MW]","Hydro Pumped Storage - Actual Consumption [MW]","Hydro Run-of-river and poundage - Actual Aggregated [MW]","Hydro Water Reservoir - Actual Aggregated [MW]","Marine - Actual Aggregated [MW]","Other - Actual Aggregated [MW]","Other renewable - Actual Aggregated [MW]","Waste - Actual Aggregated [MW]"]) for df in df_gen] 
df_gen = [rename_and_reoder_columns(df, None, ['Date', 'Nuclear', 'Solar', 'Wind Offshore', 'Wind Onshore']) for df in df_gen]
df_gen = [setup_time(df, 'Date', '%d.%m.%Y %H:%M') for df in df_gen]
df_gen = [fill_hourly_nans_by_rolling_mean(df, 'Date', 'Nuclear') for df in df_gen]
df_gen = [fill_hourly_nans_by_rolling_mean(df, 'Date', 'Solar') for df in df_gen]
df_gen = [fill_hourly_nans_by_rolling_mean(df, 'Date', 'Wind Offshore') for df in df_gen]
df_gen = [fill_hourly_nans_by_rolling_mean(df, 'Date', 'Wind Onshore') for df in df_gen]

df = concat_data(df_gen)
df_summary(df)

df.to_csv('../data/clean/exploitable/Concat_gen.csv', index=False)

OUI
OUI
OUI
OUI
Shape: (35064, 5)

Column Types:
Date             datetime64[ns]
Nuclear                 float64
Solar                   float64
Wind Offshore           float64
Wind Onshore            float64
dtype: object

Missing Values:
Date                 0
Nuclear              0
Solar                0
Wind Offshore    21432
Wind Onshore         0
dtype: int64

First Rows:
                 Date  Nuclear  Solar  Wind Offshore  Wind Onshore
0 2021-01-01 00:00:00  50378.0    0.0            NaN        1082.0
1 2021-01-01 01:00:00  49465.0    0.0            NaN        1075.0
2 2021-01-01 02:00:00  48294.0    0.0            NaN        1086.0
3 2021-01-01 03:00:00  48380.0    0.0            NaN        1149.0
4 2021-01-01 04:00:00  48560.0    0.0            NaN        1170.0

Last Rows:
                     Date    Nuclear  Solar  Wind Offshore  Wind Onshore
35059 2024-12-31 19:00:00  49890.625    0.0         83.125      1162.500
35060 2024-12-31 20:00:00  49916.750    0.0        115.125 

In [40]:
df_prices = [drop_unecessary_columns(df, ['Area', 'Sequence', 'Intraday Period (CET/CEST)', 'Intraday Price (EUR/MWh)']) for df in df_prices] 
df_prices = [rename_and_reoder_columns(df, None, ['Date', 'Day-ahead Price (EUR/MWh)']) for df in df_prices]
df_prices = [setup_time(df, 'Date', '%d/%m/%Y %H:%M:%S') for df in df_prices]
df_prices = [fill_hourly_nans_by_rolling_mean(df, 'Date', 'Day-ahead Price (EUR/MWh)') for df in df_prices]

df = concat_data(df_prices)
df_summary(df)

df.to_csv('../data/clean/exploitable/Concat_price.csv', index=False)

OUI
OUI
OUI
OUI
Shape: (35064, 2)

Column Types:
Date                         datetime64[ns]
Day-ahead Price (EUR/MWh)           float64
dtype: object

Missing Values:
Date                         0
Day-ahead Price (EUR/MWh)    0
dtype: int64

First Rows:
                 Date  Day-ahead Price (EUR/MWh)
0 2021-01-01 00:00:00                      50.87
1 2021-01-01 01:00:00                      48.19
2 2021-01-01 02:00:00                      44.68
3 2021-01-01 03:00:00                      42.92
4 2021-01-01 04:00:00                      40.39

Last Rows:
                     Date  Day-ahead Price (EUR/MWh)
35059 2024-12-31 19:00:00                      92.78
35060 2024-12-31 20:00:00                      79.04
35061 2024-12-31 21:00:00                      50.11
35062 2024-12-31 22:00:00                      63.36
35063 2024-12-31 23:00:00                      60.18
----------------------------------------
